<a href="https://colab.research.google.com/github/AndreyDyachkov/Data_analytics_with_Python/blob/main/10_yandex_zen_news_dashboard/Yandex_Zen_dashboard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Yandex.Zen dashboard

<b>Task:</b>
Get data from Yandex.Zen pipeline and build a dashboard in Tableau. Answer the questions:
1. How many user interactions with cards occur in the system, broken down by card topics?
2. How many cards do sources with different topics generate?
3. How do the topics of the cards and the topics of the sources compare?

<b>Data source:</b> an aggregate table (The dashboard will be based on a pipeline that will take data from a table that stores raw data, transform the data and put it in an aggregate table.

<b>Plan based on the results of the discussion:</b>
<br><b>Business task:</b>analysis of user interaction with Yandex.Zen cards;
<br><b>How often is it supposed to use the dashboard:</b> at least once a week;
<br><b>Who will be the main user of the dashboard:</b> content analysis managers;
<br><b>Composition of data for the dashboard:</b>
1. History of events by topic of cards (two graphs: absolute numbers and percentage);
2. Breakdown of events by topic of sources;
3. Table of correspondence between source topics and cards topics;
<br>What parameters should data be grouped by:
Date and time;
The theme of the card;
Source topic;
Age group;

<br><b>Data:</b>
- History of events by card topic - absolute values ​​broken down by minutes;
- Breakdown of events by source topic - relative values ​​(%);
- Correspondence of the source topics to the cards topics - absolute values;

<br><b>Data sources for the dashboard:</b>raw data on user interaction events with cards (log_raw table);
Database where the aggregated data will be stored: additional aggregated tables in the Zen database.

### Stage 1. Getting data

#### Imports and connection to db

In [ ]:
!pip install psycopg2

In [ ]:
import pandas as pd
from sqlalchemy import create_engine

db_config = {'user': 'praktikum_student', # login
            'pwd': 'Sdf4$2;d-d30pp', # password
            'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
            'port': 6432, # port
            'db': 'data-analyst-zen-project-db'} # db name

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                db_config['pwd'],
                                                db_config['host'],
                                                db_config['port'],
                                                db_config['db'])

engine = create_engine(connection_string)

#### SQL query

In [ ]:
query = ''' SELECT *
            FROM dash_visits
        '''

#### Writing data into a dataframe

In [ ]:
df = pd.io.sql.read_sql(query, con = engine)
df.head()

<b>Data description: </b>
We assume the following:
- record_id - id;
- item_topic - card topic;
- source_topic - source topic;
- age_segment - age category
- dt - date time
- visits - number of interactions

Let's look at the table with raw data for verification

In [ ]:
query2 = ''' SELECT *
            FROM log_raw
        '''

In [ ]:
df_raw = pd.io.sql.read_sql(query2, con = engine)
df_raw.head()

It seems that everything is correct, all interactions are taken into account in the aggregation table without breaking down by event type (show, click, etc.)

#### Checking for missing values, duplicates and wrong data types 

In [ ]:
df.info()

In [ ]:
df.duplicated().sum()

Everything looks good

### Stage 2. EDA

Descriptive statistics. We can do it in Tableau, as well. In order to avoid possible errors earlier, let's check here.

In [ ]:
df.describe()

In [ ]:
df[['item_topic','source_topic','age_segment']].describe()

In [ ]:
df['dt'].describe(datetime_is_numeric=True)

The range in the number of visits is large, the mean is much higher than the median - a right-skewed distribution. There are 25 different item_topics and 26 source_topics. Number of segments by age: 6. We have data for about half an hour. Let's look at the list of topics and age segments: for incomplete duplicates and to understand the distribution.

In [ ]:
df['item_topic'].value_counts()

In [ ]:
df['source_topic'].value_counts()

In [ ]:
df['age_segment'].value_counts().sort_index()

The number of entries for each topic is more or less equal. By age: the younger the age group, the more entries it has.

### Stage 3. Dataframe to csv

In [ ]:
df.to_csv('dash_visits.csv', index=False)

### Conclusion

We connected to the database, downloaded the aggregate table of visits to Yandex_Zen for the period: 2019-09-24 18:28:00 - 2019-09-24 19:0:00. Then we checked the data for errors and wrote it to a csv file for uploading to Tableau public.

The dashboard: https://public.tableau.com/app/profile/andrewd6063/viz/Y_project_sprint_11/Dashboard3
